In [66]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from keras.utils import plot_model
from keras.models import load_model

In [67]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/home/cheshire/NLP/data/dates/input.txt'

In [68]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
print(input_characters)
target_characters = sorted(list(target_characters))
print(target_characters)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

[' ', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\\', '_']
['\t', '\n', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Number of samples: 10000
Number of unique input tokens: 15
Number of unique output tokens: 13
Max sequence length for inputs: 10
Max sequence length for outputs: 12


In [69]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
print(len(input_texts), max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
print(len(input_texts), max_decoder_seq_length, num_decoder_tokens)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_input_data is ahead of decoder_target_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

10000 10 15
10000 12 13


In [70]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

plot_model(model, to_file='model.png', show_shapes=True)

In [74]:
# Run training
#model = load_model('s2s.h5')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 3s - loss: 0.0149 - val_loss: 0.0044
Epoch 2/100
8000/8000 [==============================] - 2s - loss: 0.0052 - val_loss: 0.0026
Epoch 3/100
8000/8000 [==============================] - 2s - loss: 0.0062 - val_loss: 0.0033
Epoch 4/100
8000/8000 [==============================] - 2s - loss: 0.0084 - val_loss: 0.0035
Epoch 5/100
8000/8000 [==============================] - 2s - loss: 0.0042 - val_loss: 0.0023
Epoch 6/100
8000/8000 [==============================] - 2s - loss: 0.0097 - val_loss: 0.0039
Epoch 7/100
8000/8000 [==============================] - 2s - loss: 0.0045 - val_loss: 0.0022
Epoch 8/100
8000/8000 [==============================] - 2s - loss: 0.0076 - val_loss: 0.0020
Epoch 9/100
8000/8000 [==============================] - 2s - loss: 0.0073 - val_loss: 0.0021
Epoch 10/100
8000/8000 [==============================] - 2s - loss: 0.0057 - val_loss: 0.0022
Epoch 11/10

8000/8000 [==============================] - 3s - loss: 0.0025 - val_loss: 0.0298
Epoch 87/100
8000/8000 [==============================] - 2s - loss: 0.0014 - val_loss: 6.6061e-04
Epoch 88/100
8000/8000 [==============================] - 3s - loss: 0.0033 - val_loss: 0.0011
Epoch 89/100
8000/8000 [==============================] - 3s - loss: 0.0022 - val_loss: 0.0034
Epoch 90/100
8000/8000 [==============================] - 2s - loss: 0.0024 - val_loss: 0.0022
Epoch 91/100
8000/8000 [==============================] - 3s - loss: 0.0014 - val_loss: 4.2335e-04
Epoch 92/100
8000/8000 [==============================] - 3s - loss: 0.0024 - val_loss: 9.7319e-04
Epoch 93/100
8000/8000 [==============================] - 3s - loss: 0.0023 - val_loss: 0.0022
Epoch 94/100
8000/8000 [==============================] - 3s - loss: 0.0012 - val_loss: 7.5526e-04
Epoch 95/100
8000/8000 [==============================] - 2s - loss: 0.0033 - val_loss: 0.0015
Epoch 96/100
8000/8000 [=======================

In [78]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

counter = 0
for seq_index in range(10000):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    if str(decoded_sentence) != str(target_texts[seq_index][1:]):
        counter += 1
        print('-')
        print('Input sentence:', input_texts[seq_index])
        print('Decoded sentence:', repr(decoded_sentence))
        print('Target sentence:', repr(target_texts[seq_index][1:]))

-
Input sentence: 1_11_1961
Decoded sentence: '01.11.1916\n'
Target sentence: '01.11.1961\n'
-
Input sentence: 8\10\4440
Decoded sentence: '08.10.4404\n'
Target sentence: '08.10.4440\n'
-
Input sentence: 27_4_2206
Decoded sentence: '27.04.2260\n'
Target sentence: '27.04.2206\n'
-
Input sentence: 05/08/2228
Decoded sentence: '05.08.2282\n'
Target sentence: '05.08.2228\n'
-
Input sentence: 27\7\6097
Decoded sentence: '27.06.7096\n'
Target sentence: '27.07.6097\n'
-
Input sentence: 16.7.1661
Decoded sentence: '16.07.6161\n'
Target sentence: '16.07.1661\n'
-
Input sentence: 01_01_4560
Decoded sentence: '01.01.4506\n'
Target sentence: '01.01.4560\n'
-
Input sentence: 04 09 9719
Decoded sentence: '04.09.7919\n'
Target sentence: '04.09.9719\n'


In [79]:
print(counter/10000)

0.0008
